# Manually modelling helicity amplitudes

The goal in this notebook is to formulate helicity amplitude model manually and reproduce the result in [Amplitude model with `ampform-dpd` notebook](ampform-dpd.ipynb).

In [ ]:
from __future__ import annotations

import logging
import os
import warnings

import graphviz
import numpy as np
import qrules
import sympy as sp
from ampform.io import aslatex, improve_latex_rendering
from ampform_dpd.adapter.qrules import normalize_state_ids, to_three_body_decay
from IPython.display import Math
from qrules.particle import Particle, Spin, create_particle, load_pdg
from sympy.physics.quantum.spin import Rotation as Wigner

STATIC_PAGE = "EXECUTE_NB" in os.environ

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.disable(logging.WARNING)
warnings.filterwarnings("ignore")
improve_latex_rendering()
particle_db = load_pdg()

# Reaction 

Firstly, we check the decay topology in strong reaction as a reminder.

In [ ]:
E_lab_gamma = 8.5
m_proton = 0.938
m_0 = np.sqrt(2 * E_lab_gamma * m_proton + m_proton**2)
m_eta = 0.548
m_pi = 0.135
m_0

In [ ]:
pgamma1 = Particle(
    name="pgamma1",
    latex=r"p\gamma (s1/2)",
    spin=0.5,
    mass=m_0,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99990,
)
pgamma2 = create_particle(
    template_particle=pgamma1,
    name="pgamma2",
    latex=R"p\gamma (s3/2)",
    spin=1.5,
    pid=pgamma1.pid + 1,
)
particle_db.update([pgamma1, pgamma2])

In [ ]:
reaction = qrules.generate_transitions(
    initial_state=("pgamma1"),
    final_state=["Lambda", "K+", "pi0"],
    allowed_interaction_types=["strong"],
    formalism="helicity",
    particle_db=particle_db,
    max_angular_momentum=4,
    max_spin_magnitude=4,
    mass_conservation_factor=0,
)
reaction = normalize_state_ids(reaction)

In [ ]:
dot = qrules.io.asdot(reaction, collapse_graphs=True)
graphviz.Source(dot)

In [ ]:
decay = to_three_body_decay(reaction.transitions)
Math(aslatex(decay, with_jp=True))

## Model implementation

Initial state:

$0: p \gamma $

Final states: 

$1: \Lambda, $

$2:K^+, $

$3:\pi^0$

Resonances:

Final states 23 -> 

$1: K^*$

Final states 31 -> 

$2: \Sigma^*$

Final states 12 -> 

$3: N^*$

### $I = |A^1 + A^2 + A^3|^2$

In [ ]:
half = sp.Rational(1, 2)

A1 = sp.IndexedBase("A^1")
A2 = sp.IndexedBase("A^2")
A3 = sp.IndexedBase("A^3")

λ0, λ1 = sp.symbols(r"\lambda_0:2", rational=True)
λ0prime, λ1prime = sp.symbols(r"\lambda'_0:2", rational=True)

zeta_1_0, zeta_1_1, zeta_2_0, zeta_2_1, zeta_3_0, zeta_3_1 = sp.symbols(
    r"\zeta_{1(1)}^0 \zeta_{1(1)}^1 \zeta_{2(1)}^0 \zeta_{2(1)}^1 \zeta_{3(1)}^0 \zeta_{3(1)}^1"
)

wigner_d_11_R1 = Wigner.d(half, λ1prime, λ1, zeta_1_1)
wigner_d_00_R1 = Wigner.d(half, λ0prime, λ0, zeta_1_0)
wigner_d_11_R2 = Wigner.d(half, λ1prime, λ1, zeta_2_1)
wigner_d_00_R2 = Wigner.d(half, λ0prime, λ0, zeta_2_0)
wigner_d_11_R3 = Wigner.d(half, λ1prime, λ1, zeta_3_1)
wigner_d_00_R3 = Wigner.d(half, λ0prime, λ0, zeta_3_0)

A1_aligned = A1[λ0prime, λ1prime] * wigner_d_11_R1 * wigner_d_00_R1
A2_aligned = A2[λ0prime, λ1prime] * wigner_d_11_R2 * wigner_d_00_R2
A3_aligned = A3[λ0prime, λ1prime] * wigner_d_11_R3 * wigner_d_00_R3

In [ ]:
sp.Sum(
    sp.Pow(
        sp.Abs(
            sp.Sum(
                A1_aligned + A2_aligned + A3_aligned,
                [λ0, -half, half],
                [λ1, -half, half],
            )
        ),
        2,
    ),
    [λ0prime, -half, half],
    [λ1prime, -half, half],
)

### $A^1 \equiv A^{K^*}$

In [ ]:
s23, m_k, gamma_k, c_k = sp.symbols(r"s_{12} m_{K^*} \Gamma_{K^*} C_{K^*}")
theta23, phi23 = sp.symbols(r"theta_23 phi_23")
lambda_R = sp.symbols(r"\lambda_R")
delta = sp.KroneckerDelta(half, lambda_R)

d_k = sp.Function(r"d_{\lambda_K^*1/2}^{1/2}")

A1_expr = sp.Sum(
    (
        gamma_k
        * m_k
        * delta
        * c_k
        * d_k(theta23)
        / (-sp.I * gamma_k * m_k + m_k**2 - s23)
    ),
    (lambda_R, -half, half),
)
A1_expr

In [ ]:
A1_expr.doit()

### $A^2 \equiv A^{\Sigma^*}$

In [ ]:
s31, m_sigma, gamma_sigma, c_sigma = sp.symbols(
    r"s_{31} m_{\Sigma^*} \Gamma_{\Sigma^*} C_{\Sigma^*}"
)
theta31, phi31, delta = sp.symbols(r"theta_31 phi_31 \delta_{\lambda_\Sigma^*1/2}")
d_s = sp.Function(r"d_{\lambda_\Sigma^*1/2}^{1/2}")

A2_expr = (
    gamma_sigma
    * m_sigma
    * delta
    * c_sigma
    * d_s(theta31)
    / (-sp.I * gamma_sigma * m_sigma + m_sigma**2 - s31)
)
A2_expr

### $A^3 \equiv A^{N^*}$

In [ ]:
s12, m_n, gamma_n, c_n = sp.symbols(r"s_{12} m_{N^*} \Gamma_{N^*} C_{N^*}")
theta12, phi12, delta = sp.symbols(r"theta_12 phi_12 \delta_{\lambda_N^*1/2}")
d_n = sp.Function(r"d_{\lambda_N^*1/2}^{1/2}")

A3_expr = (
    gamma_n * m_n * delta * c_n * d_n(theta12) / (-sp.I * gamma_n * m_n + m_n**2 - s12)
)
A3_expr